# 要約 
このJupyter Notebookは、LMSYS - Chatbot Arena コンペティションにおいて、ユーザーが好む応答を予測するための機械学習モデルを構築することを目的としています。特に、チャットボットの応答の中から、どちらがユーザーに好まれるかを判定するための特徴量を抽出し、予測モデルを訓練する方法を示しています。

### 主な問題:
- コンペティションのデータを用いて、二者間の選好（どちらの応答が好まれるか）を予測する。

### 使用している手法:
1. **データの前処理**:
   - プロンプトや応答をクリーニングし、'null'応答を持つ行を削除する。

2. **特徴量エンジニアリング**:
   - **TF-IDF**ベクトル化（単語と文字レベル）を用いてテキストデータを数値化し、各応答の特徴を抽出。
   - 応答の長さに基づく特徴を作成し、応答の長さの差や比も考慮。
   - パープレキシティを算出することで、応答の自然さを測定し、IFD（Input-Output Frequency Discrepancy）に基づいた特徴量を生成。

3. **モデルの構築**:
   - LightGBMを用いた多クラス分類器を使用、一対多の選好予測を行う。交差検証を通じて汎用性を向上させ、評価指標として対数損失（Log Loss）を使用してモデルの性能を測定。

### 使用されているライブラリ:
- **numpy, pandas**: データの操作と分析。
- **regex**: テキスト処理のための正規表現。
- **scikit-learn**: 特徴量エンジニアリングやモデル評価。
- **lightgbm**: 高速で効率的なブースティング決定木。
- **transformers**: DeBERTaとGPTモデルを活用し、自然言語の理解を深めるためのパープレキシティ計算。
- **tqdm**: 進捗バーの表示。

このノートブックは、ユーザーの選好を予測するために、データの前処理からモデル構築、予測までの一連のステップを網羅しており、特にTF-IDFやパープレキシティなどのテキスト特徴量を活用している点が特徴です。最終的に、予測された結果はCSV形式で出力され、コンペティションに提出可能な形式が整えられています。

---


# 用語概説 
以下は、提供されたJupyter Notebookに関連する機械学習・深層学習の専門用語の簡単な解説です。特に、初心者には馴染みが薄い可能性がある用語やこのノートブック特有のドメイン知識に焦点を当てています。

### 専門用語の解説

1. **IFD (Instructive Feedback Dynamics)**
   - モデルが生成した応答の質を評価するための指標。具体的には、ある質問に対する応答が、他の応答に対するパープレキシティ（PPL）とどう関連しているかを示す指標です。小さいほど良い応答とされ、ユーザーから見てどれほど好ましいかの参考になります。

2. **パープレキシティ (Perplexity)**
   - 言語モデルの性能を測るための指標。モデルが新しいデータをどれだけうまく予測できるかを示し、値が低いほど良い。わかりやすく言うと、モデルが与えられたテキストを理解し、次に来る単語を予測できる程度を示す。

3. **TF-IDF (Term Frequency-Inverse Document Frequency)**
   - 文書内の用語の重要性を評価するための統計的手法。特定の用語が文書内でどれだけ頻繁に使われているか（Term Frequency）と、全体の文書集でその用語がどれだけ珍しいか（Inverse Document Frequency）を組み合わせることで、文書の特徴を抽出します。

4. **特徴量ベクトル (Feature Vector)**
   - 機械学習モデルにおける入力データの表現。複数の特徴量（変数）を数値のリストとして表現したもので、モデルが予測を行うための基盤となります。

5. **バッチ処理 (Batch Processing)**
   - データを小さなグループ（バッチ）に分割して処理する方法。これは、メモリの消費を抑えたり、処理効率を高めるために使用されます。

6. **交差検証 (Cross Validation)**
   - モデルの汎化能力を評価するためにデータを分割して繰り返しトレーニングとテストを行う手法。これにより、特定のデータセットに対する過剰適合を防ぎます。

7. **LightGBM (Light Gradient Boosting Machine)**
   - 勾配ブースティングフレームワークの一つで、大規模なデータセットで高速に学習できるように最適化されています。特に、ツリー構造の学習において効率的で、メモリ使用量を抑える設計がされています。

8. **スパース行列 (Sparse Matrix)**
   - 値がほとんどゼロである行列のこと。計算効率を高めるために、ゼロ以外の値だけを格納するデータ構造を使用します。

9. **ハイパーパラメータ (Hyperparameter)**
   - モデルの構造や学習プロセスにおいて手動で設定するパラメータ。たとえば、決定木の深さやサンプリング率などがあります。これらは訓練データを通じて学習されるパラメータとは異なり、モデルの学習前に設定されます。

10. **エラー処理 (Error Handling)**
    - コードが実行中に発生しうるエラーに対して、プログラムがどのように対処するかを定義するプロセス。ここでは、発生したエラーに基づいて、適切な戻り値（例えば、0など）を返すような処理について言及しています。

これらの解説はノートブック全体を通じて初心者がつまずく可能性のあるポイントを考慮して作成しました。理解を深めるためにこれらの用語を活用してください。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Refer: [IFD](https://github.com/tianyi-lab/Superfiltering)
$$\mathrm{IFD}_\theta(Q,A)=\frac{\mathrm{PPL}_\theta(A|Q)}{\mathrm{PPL}_\theta(A)}$$


</div>
<div class="column-right">

# 日本語訳

## 参照: [IFD](https://github.com/tianyi-lab/Superfiltering)
$$\mathrm{IFD}_\theta(Q,A)=\frac{\mathrm{PPL}_\theta(A|Q)}{\mathrm{PPL}_\theta(A)}$$



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import numpy as np 
import pandas as pd 
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, save_npz, load_npz, hstack
import lightgbm as lgb
from tqdm import tqdm
import gensim
import itertools
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from transformers import DebertaV2Tokenizer, DebertaV2Model
from transformers import set_seed, AutoModelForCausalLM, AutoTokenizer
import torch
import joblib
import unicodedata
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
import numpy as np 
import pandas as pd 
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, save_npz, load_npz, hstack
import lightgbm as lgb
from tqdm import tqdm
import gensim
import itertools
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from transformers import DebertaV2Tokenizer, DebertaV2Model
from transformers import set_seed, AutoModelForCausalLM, AutoTokenizer
import torch
import joblib
import unicodedata
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
```

</div>
</details>

In [ ]:
import os
import numpy as np 
import pandas as pd 
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, StratifiedGroupKFold, train_test_split
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, accuracy_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, save_npz, load_npz, hstack
import lightgbm as lgb
from tqdm import tqdm
import gensim
import itertools
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
from transformers import DebertaV2Tokenizer, DebertaV2Model
from transformers import set_seed, AutoModelForCausalLM, AutoTokenizer
import torch
import joblib
import unicodedata
import re
import time
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
MAX_LENGTH = 1024
deberta_path = "/kaggle/input/debertav3base"
gpt_path = "/kaggle/input/qwen2-1-5b-instruct"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
SEED = 42
set_seed(SEED)
```

</div>
<div class="column-right">

# 日本語訳

```python
MAX_LENGTH = 1024  # モデルの最大入力長を設定
deberta_path = "/kaggle/input/debertav3base"  # DeBERTaモデルのパス
gpt_path = "/kaggle/input/qwen2-1-5b-instruct"  # GPTモデルのパス
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能か確認し、デバイスを設定
SEED = 42  # 乱数シードを設定
set_seed(SEED)  # 乱数シードを設定する関数を呼び出し
```

</div>
</details>

In [ ]:
MAX_LENGTH = 1024  # モデルの最大入力長を設定
deberta_path = "/kaggle/input/debertav3base"  # DeBERTaモデルのパス
gpt_path = "/kaggle/input/qwen2-1-5b-instruct"  # GPTモデルのパス
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPUが利用可能か確認し、デバイスを設定
SEED = 42  # 乱数シードを設定
set_seed(SEED)  # 乱数シードを設定する関数を呼び出し

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")
vectorize_on_train_and_test = True
#quick_test for training on small part of train data (and not using bunch of GPU on submit)
#(if this is on - saved models won't be fully trained)
quick_test = True
quick_test_items = 1000
#automatically disable quick_test if we detect actual test data... (assures full training when scoring)
if (len(test)) > 3:quick_test = False
if quick_test: train = train.head(quick_test_items)

def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)
print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))

target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]
train['label'] = train[target_columns].idxmax(axis=1) 
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]
train.head(3)
```

</div>
<div class="column-right">

# 日本語訳

```python
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # トレーニングデータを読み込む
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")  # テストデータを読み込む
vectorize_on_train_and_test = True  # トレーニングデータとテストデータのベクトル化を実行するかのフラグ
# トレーニングデータの小さい部分でのクイックテスト用（提出時に多くのGPUを使用しないように）
# (これがオンの場合、保存されたモデルは完全には訓練されない)
quick_test = True
quick_test_items = 1000  # クイックテストで使用するデータの数
# 実際のテストデータが検出される場合、自動的にクイックテストを無効にする...（スコアリング時に完全なトレーニングを保証する）
if (len(test)) > 3: quick_test = False
if quick_test: train = train.head(quick_test_items)  # クイックテストの場合、トレーニングデータを制限する

def process(input_str):
    stripped_str = input_str.strip('[]')  # 入力文字列の前後の中括弧を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文をリストに分割し、余分な引用符を削除
    return ' '.join(sentences)  # 文を結合して一つの文字列にする

# テストデータのプロンプト、応答A、応答Bを処理
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)
# トレーニングデータのプロンプト、応答A、応答Bを処理
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 応答Aと応答Bが両方'null'のインデックスを取得
train.drop(indexes, inplace=True)  # 'null'のインデックスをトレーニングデータから削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット
print(f"Total {len(indexes)} Null response rows dropped")  # 削除された行数を表示
print('Total train samples: ', len(train))  # トレーニングデータのサンプル数を表示

target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ターゲットとなるカラムを定義
columns_to_vectorize = ["prompt", "response_a", "response_b"]  # ベクトル化するカラムを定義
train['label'] = train[target_columns].idxmax(axis=1)  # 各行の勝者モデルをラベルとして設定
label_encoder = LabelEncoder()  # ラベルエンコーダを作成
train['label'] = label_encoder.fit_transform(train['label'])  # ラベルをエンコード
train = train[columns_to_vectorize + ['label']]  # 必要なカラムのみを保持
train.head(3)  # トレーニングデータの最初の3行を表示
```

</div>
</details>

In [ ]:
train = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # トレーニングデータを読み込む
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")  # テストデータを読み込む
vectorize_on_train_and_test = True  # トレーニングデータとテストデータのベクトル化を実行するかのフラグ
# トレーニングデータの小さい部分でのクイックテスト用（提出時に多くのGPUを使用しないように）
# (これがオンの場合、保存されたモデルは完全には訓練されない)
quick_test = True
quick_test_items = 1000  # クイックテストで使用するデータの数
# 実際のテストデータが検出される場合、自動的にクイックテストを無効にする...（スコアリング時に完全なトレーニングを保証する）
if (len(test)) > 3: quick_test = False
if quick_test: train = train.head(quick_test_items)  # クイックテストの場合、トレーニングデータを制限する

def process(input_str):
    stripped_str = input_str.strip('[]')  # 入力文字列の前後の中括弧を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 各文をリストに分割し、余分な引用符を削除
    return ' '.join(sentences)  # 文を結合して一つの文字列にする

# テストデータのプロンプト、応答A、応答Bを処理
test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)
# トレーニングデータのプロンプト、応答A、応答Bを処理
train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index  # 応答Aと応答Bが両方'null'のインデックスを取得
train.drop(indexes, inplace=True)  # 'null'のインデックスをトレーニングデータから削除
train.reset_index(inplace=True, drop=True)  # インデックスをリセット
print(f"Total {len(indexes)} Null response rows dropped")  # 削除された行数を表示
print('Total train samples: ', len(train))  # トレーニングデータのサンプル数を表示

target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']  # ターゲットとなるカラムを定義
columns_to_vectorize = ["prompt", "response_a", "response_b"]  # ベクトル化するカラムを定義
train['label'] = train[target_columns].idxmax(axis=1)  # 各行の勝者モデルをラベルとして設定
label_encoder = LabelEncoder()  # ラベルエンコーダを作成
train['label'] = label_encoder.fit_transform(train['label'])  # ラベルをエンコード
train = train[columns_to_vectorize + ['label']]  # 必要なカラムのみを保持
train.head(3)  # トレーニングデータの最初の3行を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### TF-IDF

</div>
<div class="column-right">

# 日本語訳

### TF-IDF



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train_text = train[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)
test_text = test[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)

if vectorize_on_train_and_test:
    vector_fit_text = pd.concat([train_text, test_text], axis=0).reset_index(drop=True)
else:
    vector_fit_text = train_text
```

</div>
<div class="column-right">

# 日本語訳

```python
train_text = train[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)  # トレーニングデータの全テキストを結合
test_text = test[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)  # テストデータの全テキストを結合

if vectorize_on_train_and_test:  # トレーニングとテスト両方をベクトル化する場合
    vector_fit_text = pd.concat([train_text, test_text], axis=0).reset_index(drop=True)  # トレーニングデータとテストデータを連結
else:
    vector_fit_text = train_text  # そうでなければトレーニングデータのみを使用
```

</div>
</details>

In [ ]:
train_text = train[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)  # トレーニングデータの全テキストを結合
test_text = test[['prompt', 'response_a', 'response_b']].astype(str).apply(lambda x: ' '.join(x), axis=1)  # テストデータの全テキストを結合

if vectorize_on_train_and_test:  # トレーニングとテスト両方をベクトル化する場合
    vector_fit_text = pd.concat([train_text, test_text], axis=0).reset_index(drop=True)  # トレーニングデータとテストデータを連結
else:
    vector_fit_text = train_text  # そうでなければトレーニングデータのみを使用

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def custom_tokenizer(text):
    return re.findall(r'[^\W]+', text)

#word-level vectorizer
tfidf_word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 5),
    tokenizer=custom_tokenizer,
    token_pattern=None,
    strip_accents='unicode',
    min_df=4,
    max_features=300
)

#char-level vectorizer
tfidf_char_vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1, 5), 
    max_features=1000, 
    min_df=4
)

def batch_process(texts, batch_size):
    for i in range(0, len(texts), batch_size):
        yield texts[i:i + batch_size]

#doing in batches so we can see progress
batch_size = 1000
for batch in tqdm(batch_process(vector_fit_text, batch_size), total=np.ceil(len(vector_fit_text) / batch_size)):
    if len(batch) >= tfidf_word_vectorizer.min_df:
        tfidf_word_vectorizer.fit(batch)
    if len(batch) >= tfidf_char_vectorizer.min_df:
        tfidf_char_vectorizer.fit(batch)
        
def get_tfidf_vectors(df):
    vectorized_columns = []
    for column in columns_to_vectorize:
        vectorized_columns.append(tfidf_word_vectorizer.transform(df[column]))
        vectorized_columns.append(tfidf_char_vectorizer.transform(df[column]))
    return hstack(vectorized_columns)

tfidf_train_vectors = get_tfidf_vectors(train)
```

</div>
<div class="column-right">

# 日本語訳

```python
def custom_tokenizer(text):
    return re.findall(r'[^\W]+', text)  # 特殊文字を除外して単語を抽出するカスタムトークナイザー

# word-level vectorizer
tfidf_word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 5),  # n-gramの範囲を設定（1から5）
    tokenizer=custom_tokenizer,  # カスタムトークナイザーを指定
    token_pattern=None,  # デフォルトのトークンパターンを無効に
    strip_accents='unicode',  # アクセントを削除
    min_df=4,  # 最小出現回数
    max_features=300  # 最大特徴数
)

# char-level vectorizer
tfidf_char_vectorizer = TfidfVectorizer(
    analyzer='char',  # 文字単位のベクトル化を実行
    ngram_range=(1, 5),  # n-gramの範囲
    max_features=1000,  # 最大特徴数
    min_df=4  # 最小出現回数
)

def batch_process(texts, batch_size):
    for i in range(0, len(texts), batch_size):  # バッチサイズごとにテキストを処理
        yield texts[i:i + batch_size]  # 各バッチを返す

# バッチごとに進行状況を表示しながら処理
batch_size = 1000  # バッチサイズを設定
for batch in tqdm(batch_process(vector_fit_text, batch_size), total=np.ceil(len(vector_fit_text) / batch_size)):  # 進捗バーを表示
    if len(batch) >= tfidf_word_vectorizer.min_df:  # 最小出現回数を考慮して単語ベクトルをフィット
        tfidf_word_vectorizer.fit(batch)
    if len(batch) >= tfidf_char_vectorizer.min_df:  # 最小出現回数を考慮して文字ベクトルをフィット
        tfidf_char_vectorizer.fit(batch)
        
def get_tfidf_vectors(df):
    vectorized_columns = []  # ベクトル化されたカラムのリスト
    for column in columns_to_vectorize:  # 各列に対して
        vectorized_columns.append(tfidf_word_vectorizer.transform(df[column]))  # 単語ベクトルを付加
        vectorized_columns.append(tfidf_char_vectorizer.transform(df[column]))  # 文字ベクトルを付加
    return hstack(vectorized_columns)  # スパース行列として返す

tfidf_train_vectors = get_tfidf_vectors(train)  # トレーニングデータのTF-IDFベクトルを取得
```

</div>
</details>

In [ ]:
def custom_tokenizer(text):
    return re.findall(r'[^\W]+', text)  # 特殊文字を除外して単語を抽出するカスタムトークナイザー

# word-level vectorizer
tfidf_word_vectorizer = TfidfVectorizer(
    ngram_range=(1, 5),  # n-gramの範囲を設定（1から5）
    tokenizer=custom_tokenizer,  # カスタムトークナイザーを指定
    token_pattern=None,  # デフォルトのトークンパターンを無効に
    strip_accents='unicode',  # アクセントを削除
    min_df=4,  # 最小出現回数
    max_features=300  # 最大特徴数
)

# char-level vectorizer
tfidf_char_vectorizer = TfidfVectorizer(
    analyzer='char',  # 文字単位のベクトル化を実行
    ngram_range=(1, 5),  # n-gramの範囲
    max_features=1000,  # 最大特徴数
    min_df=4  # 最小出現回数
)

def batch_process(texts, batch_size):
    for i in range(0, len(texts), batch_size):  # バッチサイズごとにテキストを処理
        yield texts[i:i + batch_size]  # 各バッチを返す

# バッチごとに進行状況を表示しながら処理
batch_size = 1000  # バッチサイズを設定
for batch in tqdm(batch_process(vector_fit_text, batch_size), total=np.ceil(len(vector_fit_text) / batch_size)):  # 進捗バーを表示
    if len(batch) >= tfidf_word_vectorizer.min_df:  # 最小出現回数を考慮して単語ベクトルをフィット
        tfidf_word_vectorizer.fit(batch)
    if len(batch) >= tfidf_char_vectorizer.min_df:  # 最小出現回数を考慮して文字ベクトルをフィット
        tfidf_char_vectorizer.fit(batch)
        
def get_tfidf_vectors(df):
    vectorized_columns = []  # ベクトル化されたカラムのリスト
    for column in columns_to_vectorize:  # 各列に対して
        vectorized_columns.append(tfidf_word_vectorizer.transform(df[column]))  # 単語ベクトルを付加
        vectorized_columns.append(tfidf_char_vectorizer.transform(df[column]))  # 文字ベクトルを付加
    return hstack(vectorized_columns)  # スパース行列として返す

tfidf_train_vectors = get_tfidf_vectors(train)  # トレーニングデータのTF-IDFベクトルを取得

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### LEN

</div>
<div class="column-right">

# 日本語訳

### LEN



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def has_none(vals) -> int:
    # some responses contains null and probably they are useful for prediction
    return int(any(val is None for val in vals))

def str_length(vals) -> int:
    length = 0
    for val in vals:
        if isinstance(val, str):
            length += len(val)
    return length

def get_length_features(data: pd.DataFrame):
    length_feature_array = []
    length_feature_array.append(data["response_a"].apply(str_length))
    length_feature_array.append(data["response_b"].apply(str_length))
    length_feature_array.append(length_feature_array[0] - length_feature_array[1])
    length_feature_array.append((length_feature_array[0] + length_feature_array[1]) / 2)
    length_feature_array.append((length_feature_array[0] / length_feature_array[1]))
    length_feature_array.append(data["response_a"].apply(has_none))
    length_feature_array.append(data["response_b"].apply(has_none))
    length_feature_array.append(data["response_a"].apply(has_none) - data["response_b"].apply(has_none))
    length_feature_array = np.array(length_feature_array).reshape(len(length_feature_array), -1)
    length_feature_array = np.transpose(length_feature_array, (1, 0))
    return length_feature_array
train_length_features = get_length_features(train)
print(train_length_features.shape)
```

</div>
<div class="column-right">

# 日本語訳

```python
def has_none(vals) -> int:
    # 応答にnullが含まれる場合、予測において役立つ可能性がある
    return int(any(val is None for val in vals))  # 一つでもnullがあれば1を返す

def str_length(vals) -> int:
    length = 0  # 文字列の長さを保持する変数
    for val in vals:  # 各値に対して
        if isinstance(val, str):  # 値が文字列であれば
            length += len(val)  # その長さを加算
    return length  # 総長を返す

def get_length_features(data: pd.DataFrame):
    length_feature_array = []  # 長さ特徴の配列を作成
    length_feature_array.append(data["response_a"].apply(str_length))  # 応答Aの長さを取得
    length_feature_array.append(data["response_b"].apply(str_length))  # 応答Bの長さを取得
    length_feature_array.append(length_feature_array[0] - length_feature_array[1])  # 応答Aと応答Bの長さの差
    length_feature_array.append((length_feature_array[0] + length_feature_array[1]) / 2)  # 応答の平均長
    length_feature_array.append((length_feature_array[0] / length_feature_array[1]))  # 応答Aと応答Bの長さの比
    length_feature_array.append(data["response_a"].apply(has_none))  # 応答Aにnullが含まれているか
    length_feature_array.append(data["response_b"].apply(has_none))  # 応答Bにnullが含まれているか
    length_feature_array.append(data["response_a"].apply(has_none) - data["response_b"].apply(has_none))  # nullの差
    length_feature_array = np.array(length_feature_array).reshape(len(length_feature_array), -1)  # 配列を整形
    length_feature_array = np.transpose(length_feature_array, (1, 0))  # 行と列を入れ替え
    return length_feature_array  # 最終結果を返す

train_length_features = get_length_features(train)  # トレーニングデータの長さ特徴を取得
print(train_length_features.shape)  # 形状を表示
```

</div>
</details>

In [ ]:
def has_none(vals) -> int:
    # 応答にnullが含まれる場合、予測において役立つ可能性がある
    return int(any(val is None for val in vals))  # 一つでもnullがあれば1を返す

def str_length(vals) -> int:
    length = 0  # 文字列の長さを保持する変数
    for val in vals:  # 各値に対して
        if isinstance(val, str):  # 値が文字列であれば
            length += len(val)  # その長さを加算
    return length  # 総長を返す

def get_length_features(data: pd.DataFrame):
    length_feature_array = []  # 長さ特徴の配列を作成
    length_feature_array.append(data["response_a"].apply(str_length))  # 応答Aの長さを取得
    length_feature_array.append(data["response_b"].apply(str_length))  # 応答Bの長さを取得
    length_feature_array.append(length_feature_array[0] - length_feature_array[1])  # 応答Aと応答Bの長さの差
    length_feature_array.append((length_feature_array[0] + length_feature_array[1]) / 2)  # 応答の平均長
    length_feature_array.append((length_feature_array[0] / length_feature_array[1]))  # 応答Aと応答Bの長さの比
    length_feature_array.append(data["response_a"].apply(has_none))  # 応答Aにnullが含まれているか
    length_feature_array.append(data["response_b"].apply(has_none))  # 応答Bにnullが含まれているか
    length_feature_array.append(data["response_a"].apply(has_none) - data["response_b"].apply(has_none))  # nullの差
    length_feature_array = np.array(length_feature_array).reshape(len(length_feature_array), -1)  # 配列を整形
    length_feature_array = np.transpose(length_feature_array, (1, 0))  # 行と列を入れ替え
    return length_feature_array  # 最終結果を返す

train_length_features = get_length_features(train)  # トレーニングデータの長さ特徴を取得
print(train_length_features.shape)  # 形状を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### IFD

</div>
<div class="column-right">

# 日本語訳

### IFD



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def get_ifd_features(data: pd.DataFrame):
    model = AutoModelForCausalLM.from_pretrained(gpt_path, torch_dtype=torch.bfloat16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(gpt_path)
    model.eval()
    def get_ppl_features(output, instruct=''):
        try:
            answer_start_index = 0
            if instruct != '':
                answer_start_index = tokenizer.encode(instruct, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).shape[1]
            input_ids = tokenizer.encode(instruct + output, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(device)
            labels = input_ids.clone()
            labels[:, :answer_start_index] = -100
            with torch.no_grad():
                outputs = model(input_ids, labels=labels)
            perplexity = torch.exp(outputs.loss)
            return perplexity.to('cpu').item()
        except:
            return 0
        
    ppl_ifd_feature_array = []
    for i in tqdm(range(len(data)), desc="Scoring ifd"):
        instruct = data['prompt'][i]
        output_a = data['response_a'][i]
        output_b = data['response_b'][i]
        
        ppl_ca_a, ppl_da_a = get_ppl_features(output_a, instruct), get_ppl_features(output_a)
        ppl_ca_b, ppl_da_b = get_ppl_features(output_b, instruct), get_ppl_features(output_b)
        try:
            ifd_a = ppl_ca_a / ppl_da_a
        except ZeroDivisionError:
            ifd_a = 0
        try:
            ifd_b = ppl_ca_b / ppl_da_b
        except ZeroDivisionError:
            ifd_b = 0
        ppl_ifd_feature_array.append([ppl_ca_a, ppl_da_a, ifd_a, ppl_ca_b, ppl_da_b, ifd_b, ifd_a - ifd_b, ifd_a - ifd_b > 0])
    ppl_ifd_feature_array = np.array(ppl_ifd_feature_array).reshape(len(ppl_ifd_feature_array), -1)
    return ppl_ifd_feature_array
ppl_ifd_features = get_ifd_features(train)
print(ppl_ifd_features.shape)
```

</div>
<div class="column-right">

# 日本語訳

```python
def get_ifd_features(data: pd.DataFrame):
    model = AutoModelForCausalLM.from_pretrained(gpt_path, torch_dtype=torch.bfloat16).to(device)  # GPTモデルを読み込んでデバイスに移動
    tokenizer = AutoTokenizer.from_pretrained(gpt_path)  # トークナイザーを読み込む
    model.eval()  # 評価モードに設定
    
    def get_ppl_features(output, instruct=''):
        try:
            answer_start_index = 0  # 応答の開始位置を初期化
            if instruct != '':  # 指示があれば
                answer_start_index = tokenizer.encode(instruct, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).shape[1]  # 指示のトークン数を取得
            input_ids = tokenizer.encode(instruct + output, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(device)  # 入力IDを生成
            labels = input_ids.clone()  # ラベルをクローン
            labels[:, :answer_start_index] = -100  # 指示部分のラベルを無視
            with torch.no_grad():  # 勾配計算を無視
                outputs = model(input_ids, labels=labels)  # モデルへの入力とラベルを指定して出力を取得
            perplexity = torch.exp(outputs.loss)  # パープレキシティを計算
            return perplexity.to('cpu').item()  # CPUに戻して値を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    ppl_ifd_feature_array = []  # 特徴の配列を初期化
    for i in tqdm(range(len(data)), desc="Scoring ifd"):  # データに対して進捗を表示しながらループ
        instruct = data['prompt'][i]  # プロンプトを取得
        output_a = data['response_a'][i]  # 応答Aを取得
        output_b = data['response_b'][i]  # 応答Bを取得
        
        # 応答Aのパープレキシティを計算
        ppl_ca_a, ppl_da_a = get_ppl_features(output_a, instruct), get_ppl_features(output_a)
        # 応答Bのパープレキシティを計算
        ppl_ca_b, ppl_da_b = get_ppl_features(output_b, instruct), get_ppl_features(output_b)
        try:
            ifd_a = ppl_ca_a / ppl_da_a  # 応答AのIFDを計算
        except ZeroDivisionError:
            ifd_a = 0  # ゼロ除算が発生した場合は0とする
        try:
            ifd_b = ppl_ca_b / ppl_da_b  # 応答BのIFDを計算
        except ZeroDivisionError:
            ifd_b = 0  # ゼロ除算が発生した場合は0とする
        ppl_ifd_feature_array.append([ppl_ca_a, ppl_da_a, ifd_a, ppl_ca_b, ppl_da_b, ifd_b, ifd_a - ifd_b, ifd_a - ifd_b > 0])  # 特徴を追加
        
    ppl_ifd_feature_array = np.array(ppl_ifd_feature_array).reshape(len(ppl_ifd_feature_array), -1)  # 特徴の配列を整形
    return ppl_ifd_feature_array  # 結果を返す

ppl_ifd_features = get_ifd_features(train)  # トレーニングデータのIFD特徴を取得
print(ppl_ifd_features.shape)  # 形状を表示
```

</div>
</details>

In [ ]:
def get_ifd_features(data: pd.DataFrame):
    model = AutoModelForCausalLM.from_pretrained(gpt_path, torch_dtype=torch.bfloat16).to(device)  # GPTモデルを読み込んでデバイスに移動
    tokenizer = AutoTokenizer.from_pretrained(gpt_path)  # トークナイザーを読み込む
    model.eval()  # 評価モードに設定
    
    def get_ppl_features(output, instruct=''):
        try:
            answer_start_index = 0  # 応答の開始位置を初期化
            if instruct != '':  # 指示があれば
                answer_start_index = tokenizer.encode(instruct, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).shape[1]  # 指示のトークン数を取得
            input_ids = tokenizer.encode(instruct + output, return_tensors="pt", truncation=True, max_length=MAX_LENGTH).to(device)  # 入力IDを生成
            labels = input_ids.clone()  # ラベルをクローン
            labels[:, :answer_start_index] = -100  # 指示部分のラベルを無視
            with torch.no_grad():  # 勾配計算を無視
                outputs = model(input_ids, labels=labels)  # モデルへの入力とラベルを指定して出力を取得
            perplexity = torch.exp(outputs.loss)  # パープレキシティを計算
            return perplexity.to('cpu').item()  # CPUに戻して値を返す
        except:
            return 0  # エラーが発生した場合は0を返す
        
    ppl_ifd_feature_array = []  # 特徴の配列を初期化
    for i in tqdm(range(len(data)), desc="Scoring ifd"):  # データに対して進捗を表示しながらループ
        instruct = data['prompt'][i]  # プロンプトを取得
        output_a = data['response_a'][i]  # 応答Aを取得
        output_b = data['response_b'][i]  # 応答Bを取得
        
        # 応答Aのパープレキシティを計算
        ppl_ca_a, ppl_da_a = get_ppl_features(output_a, instruct), get_ppl_features(output_a)
        # 応答Bのパープレキシティを計算
        ppl_ca_b, ppl_da_b = get_ppl_features(output_b, instruct), get_ppl_features(output_b)
        try:
            ifd_a = ppl_ca_a / ppl_da_a  # 応答AのIFDを計算
        except ZeroDivisionError:
            ifd_a = 0  # ゼロ除算が発生した場合は0とする
        try:
            ifd_b = ppl_ca_b / ppl_da_b  # 応答BのIFDを計算
        except ZeroDivisionError:
            ifd_b = 0  # ゼロ除算が発生した場合は0とする
        ppl_ifd_feature_array.append([ppl_ca_a, ppl_da_a, ifd_a, ppl_ca_b, ppl_da_b, ifd_b, ifd_a - ifd_b, ifd_a - ifd_b > 0])  # 特徴を追加
        
    ppl_ifd_feature_array = np.array(ppl_ifd_feature_array).reshape(len(ppl_ifd_feature_array), -1)  # 特徴の配列を整形
    return ppl_ifd_feature_array  # 結果を返す

ppl_ifd_features = get_ifd_features(train)  # トレーニングデータのIFD特徴を取得
print(ppl_ifd_features.shape)  # 形状を表示

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

### Combine

</div>
<div class="column-right">

# 日本語訳

### Combine



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
tfidf_train_vectors = csr_matrix(tfidf_train_vectors)*0.2
ppl_ifd_features_csr = csr_matrix(ppl_ifd_features)*0.7
train_length_features_csr = csr_matrix(train_length_features)*0.1
combined_train_vectors = hstack([tfidf_train_vectors, train_length_features_csr, ppl_ifd_features_csr])
print(combined_train_vectors.shape)
print("Vectorizing test text...")
tfidf_test_vectors = get_tfidf_vectors(test)
tfidf_test_vectors_csr = csr_matrix(tfidf_test_vectors)*0.2
test_ppl_ifd_features = get_ifd_features(test)
test_ppl_ifd_features_csr = csr_matrix(test_ppl_ifd_features)*0.7
test_length_features = get_length_features(test)
test_length_features_csr = csr_matrix(test_length_features)*0.1
combined_test_vectors = hstack([tfidf_test_vectors_csr, test_length_features_csr, test_ppl_ifd_features_csr]) 
print("Done!")
```

</div>
<div class="column-right">

# 日本語訳

```python
tfidf_train_vectors = csr_matrix(tfidf_train_vectors)*0.2  # TF-IDFベクトルをスケーリング
ppl_ifd_features_csr = csr_matrix(ppl_ifd_features)*0.7  # IFD特徴をスケーリング
train_length_features_csr = csr_matrix(train_length_features)*0.1  # 長さ特徴をスケーリング
combined_train_vectors = hstack([tfidf_train_vectors, train_length_features_csr, ppl_ifd_features_csr])  # ベクトルを結合
print(combined_train_vectors.shape)  # 結合後の形状を表示
print("Vectorizing test text...")  # テストテキストをベクトル化中...
tfidf_test_vectors = get_tfidf_vectors(test)  # テストデータのTF-IDFベクトルを取得
tfidf_test_vectors_csr = csr_matrix(tfidf_test_vectors)*0.2  # テストデータのTF-IDFベクトルをスケーリング
test_ppl_ifd_features = get_ifd_features(test)  # テストデータのIFD特徴を取得
test_ppl_ifd_features_csr = csr_matrix(test_ppl_ifd_features)*0.7  # テストデータのIFD特徴をスケーリング
test_length_features = get_length_features(test)  # テストデータの長さ特徴を取得
test_length_features_csr = csr_matrix(test_length_features)*0.1  # テストデータの長さ特徴をスケーリング
combined_test_vectors = hstack([tfidf_test_vectors_csr, test_length_features_csr, test_ppl_ifd_features_csr])  # テストデータのベクトルを結合
print("Done!")  # 完了メッセージを表示
```

</div>
</details>

In [ ]:
tfidf_train_vectors = csr_matrix(tfidf_train_vectors)*0.2  # TF-IDFベクトルをスケーリング
ppl_ifd_features_csr = csr_matrix(ppl_ifd_features)*0.7  # IFD特徴をスケーリング
train_length_features_csr = csr_matrix(train_length_features)*0.1  # 長さ特徴をスケーリング
combined_train_vectors = hstack([tfidf_train_vectors, train_length_features_csr, ppl_ifd_features_csr])  # ベクトルを結合
print(combined_train_vectors.shape)  # 結合後の形状を表示
print("Vectorizing test text...")  # テストテキストをベクトル化中...
tfidf_test_vectors = get_tfidf_vectors(test)  # テストデータのTF-IDFベクトルを取得
tfidf_test_vectors_csr = csr_matrix(tfidf_test_vectors)*0.2  # テストデータのTF-IDFベクトルをスケーリング
test_ppl_ifd_features = get_ifd_features(test)  # テストデータのIFD特徴を取得
test_ppl_ifd_features_csr = csr_matrix(test_ppl_ifd_features)*0.7  # テストデータのIFD特徴をスケーリング
test_length_features = get_length_features(test)  # テストデータの長さ特徴を取得
test_length_features_csr = csr_matrix(test_length_features)*0.1  # テストデータの長さ特徴をスケーリング
combined_test_vectors = hstack([tfidf_test_vectors_csr, test_length_features_csr, test_ppl_ifd_features_csr])  # テストデータのベクトルを結合
print("Done!")  # 完了メッセージを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import joblib

max_estimators = 1000
early_stopping_limit = 100

# Data preparation
X = combined_train_vectors
y_encoded = train['label'].values

# LightGBM parameters
params = {
    'n_estimators': max_estimators,
    'max_depth': 4,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'random_state': 42,
    'learning_rate': 0.03,
    'verbose': -1  # keep logs quiet
}

# Create the model
model = lgb.LGBMClassifier(**params)

# 5-fold cross-validation
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
logloss_scores = []
accuracy_scores = []
test_pred_list = []

for fold, (train_indices, val_indices) in enumerate(stratified_k_fold.split(X, y_encoded)):
    print(f"\nFold {fold + 1}")
    X_train_fold, X_val_fold = X[train_indices], X[val_indices]
    y_train_fold, y_val_fold = y_encoded[train_indices], y_encoded[val_indices]

    def callback(env):
        if env.iteration % 10 == 0: print ("Iteration:", env.iteration, "\tLog Loss:", env.evaluation_result_list[0][2])

    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_limit), callback]
    )

    y_pred_proba_fold = model.predict_proba(X_val_fold)
    logloss_fold = log_loss(y_val_fold, y_pred_proba_fold)
    logloss_scores.append(logloss_fold)
    print(f"Log Loss: {logloss_fold}")
    
    y_pred_fold = np.argmax(y_pred_proba_fold, axis=1)
    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)
    accuracy_scores.append(accuracy_fold)
    print(f"Accuracy: {accuracy_fold}")

    test_pred_list.append(model.predict_proba(combined_test_vectors[-test.shape[0]:]))

# Calculate and print average scores
average_logloss = np.mean(logloss_scores)
average_accuracy = np.mean(accuracy_scores)
print(f"\nAverage Log Loss: {average_logloss}")
print(f"Average Accuracy: {average_accuracy}")
```

</div>
<div class="column-right">

# 日本語訳

```python
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import joblib

max_estimators = 1000  # 最大推定器の数を設定
early_stopping_limit = 100  # 早期停止の制限を設定

# データ準備
X = combined_train_vectors  # 特徴ベクトル
y_encoded = train['label'].values  # ラベルを取得

# LightGBMのパラメータ
params = {
    'n_estimators': max_estimators,  # 推定器の数
    'max_depth': 4,  # 最大深さ
    'subsample': 0.8,  # サンプリングフラクション
    'colsample_bytree': 0.8,  # 特徴のサンプリングフラクション
    'objective': 'multiclass',  # 目的関数
    'num_class': 3,  # クラス数
    'metric': 'multi_logloss',  # 評価指標
    'random_state': 42,  # 乱数シード
    'learning_rate': 0.03,  # 学習率
    'verbose': -1  # ログの出力を抑制
}

# モデルの作成
model = lgb.LGBMClassifier(**params)  # LightGBMクラス分類器を生成

# 5フォールドの交差検証
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # StratifiedKFoldを設定
logloss_scores = []  # loglossスコアを保持するリスト
accuracy_scores = []  # 精度スコアを保持するリスト
test_pred_list = []  # テストの予測結果を保持するリスト

for fold, (train_indices, val_indices) in enumerate(stratified_k_fold.split(X, y_encoded)):  # 各フォールドに対して
    print(f"\nFold {fold + 1}")  # フォールドの番号を表示
    X_train_fold, X_val_fold = X[train_indices], X[val_indices]  # フォールドのトレーニングデータとバリデーションデータ
    y_train_fold, y_val_fold = y_encoded[train_indices], y_encoded[val_indices]  # フォールドのトレーニングラベルとバリデーションラベル

    def callback(env):
        if env.iteration % 10 == 0: print("Iteration:", env.iteration, "\tLog Loss:", env.evaluation_result_list[0][2])  # ログ出力のコールバック関数

    model.fit(
        X_train_fold, y_train_fold,  # トレーニングデータとトレーニングラベルでフィット
        eval_set=[(X_val_fold, y_val_fold)],  # バリデーションデータを指定
        eval_metric='multi_logloss',  # 評価指標を指定
        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_limit), callback]  # 早期停止とコールバックを設定
    )

    y_pred_proba_fold = model.predict_proba(X_val_fold)  # バリデーションデータに対する予測確率
    logloss_fold = log_loss(y_val_fold, y_pred_proba_fold)  # loglossを計算
    logloss_scores.append(logloss_fold)  # スコアを追加
    print(f"Log Loss: {logloss_fold}")  # loglossを表示
    
    y_pred_fold = np.argmax(y_pred_proba_fold, axis=1)  # 最大の予測確率を持つクラスを取得
    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)  # 精度を計算
    accuracy_scores.append(accuracy_fold)  # スコアを追加
    print(f"Accuracy: {accuracy_fold}")  # 精度を表示

    test_pred_list.append(model.predict_proba(combined_test_vectors[-test.shape[0]:]))  # テストデータの予測確率を追加

# 平均スコアを計算して表示
average_logloss = np.mean(logloss_scores)  # 平均loglossを計算
average_accuracy = np.mean(accuracy_scores)  # 平均精度を計算
print(f"\nAverage Log Loss: {average_logloss}")  # 平均loglossを表示
print(f"Average Accuracy: {average_accuracy}")  # 平均精度を表示
```

</div>
</details>

In [ ]:
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import joblib

max_estimators = 1000  # 最大推定器の数を設定
early_stopping_limit = 100  # 早期停止の制限を設定

# データ準備
X = combined_train_vectors  # 特徴ベクトル
y_encoded = train['label'].values  # ラベルを取得

# LightGBMのパラメータ
params = {
    'n_estimators': max_estimators,  # 推定器の数
    'max_depth': 4,  # 最大深さ
    'subsample': 0.8,  # サンプリングフラクション
    'colsample_bytree': 0.8,  # 特徴のサンプリングフラクション
    'objective': 'multiclass',  # 目的関数
    'num_class': 3,  # クラス数
    'metric': 'multi_logloss',  # 評価指標
    'random_state': 42,  # 乱数シード
    'learning_rate': 0.03,  # 学習率
    'verbose': -1  # ログの出力を抑制
}

# モデルの作成
model = lgb.LGBMClassifier(**params)  # LightGBMクラス分類器を生成

# 5フォールドの交差検証
stratified_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # StratifiedKFoldを設定
logloss_scores = []  # loglossスコアを保持するリスト
accuracy_scores = []  # 精度スコアを保持するリスト
test_pred_list = []  # テストの予測結果を保持するリスト

for fold, (train_indices, val_indices) in enumerate(stratified_k_fold.split(X, y_encoded)):  # 各フォールドに対して
    print(f"\nFold {fold + 1}")  # フォールドの番号を表示
    X_train_fold, X_val_fold = X[train_indices], X[val_indices]  # フォールドのトレーニングデータとバリデーションデータ
    y_train_fold, y_val_fold = y_encoded[train_indices], y_encoded[val_indices]  # フォールドのトレーニングラベルとバリデーションラベル

    def callback(env):
        if env.iteration % 10 == 0: print("Iteration:", env.iteration, "\tLog Loss:", env.evaluation_result_list[0][2])  # ログ出力のコールバック関数

    model.fit(
        X_train_fold, y_train_fold,  # トレーニングデータとトレーニングラベルでフィット
        eval_set=[(X_val_fold, y_val_fold)],  # バリデーションデータを指定
        eval_metric='multi_logloss',  # 評価指標を指定
        callbacks=[lgb.early_stopping(stopping_rounds=early_stopping_limit), callback]  # 早期停止とコールバックを設定
    )

    y_pred_proba_fold = model.predict_proba(X_val_fold)  # バリデーションデータに対する予測確率
    logloss_fold = log_loss(y_val_fold, y_pred_proba_fold)  # loglossを計算
    logloss_scores.append(logloss_fold)  # スコアを追加
    print(f"Log Loss: {logloss_fold}")  # loglossを表示
    
    y_pred_fold = np.argmax(y_pred_proba_fold, axis=1)  # 最大の予測確率を持つクラスを取得
    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)  # 精度を計算
    accuracy_scores.append(accuracy_fold)  # スコアを追加
    print(f"Accuracy: {accuracy_fold}")  # 精度を表示

    test_pred_list.append(model.predict_proba(combined_test_vectors[-test.shape[0]:]))  # テストデータの予測確率を追加

# 平均スコアを計算して表示
average_logloss = np.mean(logloss_scores)  # 平均loglossを計算
average_accuracy = np.mean(accuracy_scores)  # 平均精度を計算
print(f"\nAverage Log Loss: {average_logloss}")  # 平均loglossを表示
print(f"Average Accuracy: {average_accuracy}")  # 平均精度を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
preds_test = np.mean(test_pred_list, axis=0)
submission = pd.DataFrame({
    'id': test["id"],
    'winner_model_a': preds_test[:, 0],
    'winner_model_b': preds_test[:, 1], 
    'winner_tie': preds_test[:, 2]
})
submission.to_csv('submission.csv', index=False)
display(submission)
```

</div>
<div class="column-right">

# 日本語訳

```python
preds_test = np.mean(test_pred_list, axis=0)  # テスト予測の平均を計算
submission = pd.DataFrame({
    'id': test["id"],  # テストデータのID
    'winner_model_a': preds_test[:, 0],  # モデルAの勝者予測
    'winner_model_b': preds_test[:, 1],  # モデルBの勝者予測
    'winner_tie': preds_test[:, 2]  # タイの勝者予測
})
submission.to_csv('submission.csv', index=False)  # 提出ファイルをCSVとして保存
display(submission)  # 提出内容を表示
```

</div>
</details>

In [ ]:
preds_test = np.mean(test_pred_list, axis=0)  # テスト予測の平均を計算
submission = pd.DataFrame({
    'id': test["id"],  # テストデータのID
    'winner_model_a': preds_test[:, 0],  # モデルAの勝者予測
    'winner_model_b': preds_test[:, 1],  # モデルBの勝者予測
    'winner_tie': preds_test[:, 2]  # タイの勝者予測
})
submission.to_csv('submission.csv', index=False)  # 提出ファイルをCSVとして保存
display(submission)  # 提出内容を表示